# Explanation

The HSC data is too large to store as one sqlite database file using github.  So instead, it needs to be fetched by the user, separately from cloning the repository. This notebook is a work-in-progress to help automate that process, and make sure that the final schema is correct.


## Sending the query
The HSC data release site provides a [command line tool](https://hsc-gitlab.mtk.nao.ac.jp/snippets/17) for querying the database; I've adapted it to run programmatically from within a python session.  Check it out; it's the file `hsc_release_query.py`.  There's a working example of a simple query in `sql_tester.ipynb`. This notebook rolls everything together: querying the server, and combining the subsets into one table.

## What gets saved?
This comes in two parts:

 1) Get the main HSC table (position, fluxes, flags for each object)
 
 2) Get a list of matched spec-z's

# Code

 **Remember to set your credentials within `hsc_credentials.py` !**

In [ ]:
from __future__ import division, print_function

# give access to importing dwarfz
import os, sys
dwarfz_package_dir = os.getcwd().split("dwarfz")[0]
if dwarfz_package_dir not in sys.path:
    sys.path.insert(0, dwarfz_package_dir)

import dwarfz
from dwarfz.hsc_credentials import credential
from dwarfz.hsc_release_query import query_wrapper
    
# back to regular import statements

import os, sys
import shutil
import glob
import pandas as pd



# Get HSC Fluxes

## Build the query
Right now it only gets the *fluxes*, not the magnitudes. So far, I haven't needed the zeropoint. But this is a good start place if you need to build a query that gets the magnitudes.

In [ ]:
sql_base = """
SELECT 
    object_id, 
    ra, dec, 
    detect_is_patch_inner, detect_is_tract_inner, detect_is_primary,
    gcmodel_flux, gcmodel_flux_err, gcmodel_flux_flags,
    rcmodel_flux, rcmodel_flux_err, rcmodel_flux_flags,
    icmodel_flux, icmodel_flux_err, icmodel_flux_flags,
    zcmodel_flux, zcmodel_flux_err, zcmodel_flux_flags,
    ycmodel_flux, ycmodel_flux_err, ycmodel_flux_flags
FROM 
    pdr1_cosmos_widedepth_median.forced
LIMIT 
    {}
OFFSET 
    {}
"""

## Make the query
**The total number of objects is currently hardcoded! Make sure this hasn't changed!**
The cleaner way to do this would be to make a simple query to the database, then count the number of records. But for now, hardcoding it is simpler.

In [ ]:
n_objects = 1263503

block_size = 250000
n_blocks = (n_objects // block_size) + 1

In [ ]:
temp_hsc_table_dir = "partial_hsc_tables"
if not os.path.exists(temp_hsc_table_dir):
    os.mkdir(temp_hsc_table_dir)

In [ ]:
limit = block_size

preview_results = False
delete_job = True
out_format = "sqlite3"

for i in range(n_blocks):
    offset = i*block_size
    
    sql = sql_base.format(limit, offset)
    
    output_filename = os.path.join(temp_hsc_table_dir, "tmp_{}.sqlite3".format(i))
    
    print(" ---------------- QUERY {} -------------------- ".format(i+1))
    print(sql)

    with open(output_filename, mode="wb") as output_file:
        query_wrapper(credential, sql, preview_results, delete_job, 
                      out_format, output_file,
                      nomail=True)

## Check if it worked

In [ ]:
database_filenames = sorted(glob.glob(os.path.join(temp_hsc_table_dir, "tmp_*.sqlite3")))
database_filenames

## Combine databases

In [ ]:
dfs = [pd.read_sql_table("table_1", "sqlite:///{}".format(database_filename),
                         index_col="object_id")
       for database_filename in database_filenames]
assert(sum(df.shape[0] for df in dfs) == n_objects)

combined = pd.concat(dfs)
assert(combined.shape[0] == n_objects)

del dfs
combined.head()


In [ ]:
for filename in database_filenames:
    os.remove(filename)

In [ ]:
combined.keys()

In [ ]:
hsc_database_filename = "HSC_COSMOS_median_forced.sqlite3"
hsc_database_filename_old = hsc_database_filename + ".old"

if os.path.exists(hsc_database_filename):
    try:
        shutil.move(hsc_database_filename, hsc_database_filename_old)
        combined.to_sql("hsc", "sqlite:///{}".format(hsc_database_filename))
    except:
        # in case there's an error during writing, don't overwrite/delete the existing database
        shutil.move(hsc_database_filename_old, hsc_database_filename)
        raise
    else:
        # only delete if combining went successfully
        os.remove(hsc_database_filename + ".old")


# Get spec-z's matched to HSC objects

## Build the query

In [ ]:
redshifts_sql = """
SELECT 
    object_id, specz_id,
    d_pos, 
    specz_ra, specz_dec,
    specz_redshift, specz_redshift_err, specz_flag_homogeneous
FROM
    pdr1_cosmos_widedepth_median.specz
"""

## Make the query

In [ ]:
preview_results = False
delete_job = True
out_format = "sqlite3"

output_filename = "specz.{}".format(out_format)
print(output_filename)

with open(output_filename, mode="wb") as output_file:
    query_wrapper(credential, redshifts_sql, preview_results, delete_job, 
                  out_format, output_file,
                  nomail=True,
                 )

## Check if it worked

In [ ]:
!ls -lh specz.sqlite3

In [ ]:
df = pd.read_sql_table("table_1", 
                            "sqlite:///{}".format("specz.sqlite3"), 
                            index_col="object_id")
df = df[df.specz_flag_homogeneous]

df.head()